<a href="https://colab.research.google.com/github/gaallmin/Prediction-of-Agricultural-Product-prices/blob/yeji/train_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Dacon

Mounted at /content/drive/
/content/drive/MyDrive/Dacon


In [6]:
import pandas as pd

train_df = pd.read_csv(r"dataview\data\train\평년평균_업데이트.csv", encoding="utf-8")

In [7]:
import numpy as np

train_df['year'] = train_df['시점'].str[:4].astype(int)
train_df['month'] = train_df['시점'].str[4:6].astype(int)
train_df['순'] = train_df['시점'].str[6:8]
train_df = train_df[['year', 'month', '순'] + [col for col in train_df.columns if col not in ['year', 'month', '순']]]

In [8]:
import numpy as np

# 1. '평년 평균가격(원)' 컬럼에 로그 변환 적용
train_df['평년 평균가격(원)_로그'] = np.log(train_df['평년 평균가격(원)'].replace(0, np.nan))  # 로그는 0에 대해 정의되지 않으므로 0을 NaN으로 처리
train_df['평년 평균가격(원)_로그'] = train_df['평년 평균가격(원)_로그'].fillna(0)

train_df['평균가격(원)_로그'] = np.log(train_df['평균가격(원)'].replace(0, np.nan))
train_df['평균가격(원)_로그'] = train_df['평균가격(원)_로그'].fillna(0)

In [9]:
import pandas as pd

# 품종 쌍과 거래단위를 설정
product_pairs = [
    ('양파', '20키로', '양파', '10키로'),
    ('양파', '12키로', '양파(햇)', '12키로'),
    ('양파', '15키로', '양파(햇)', '15키로'),
    ('양파', '20키로', '양파(햇)', '20키로'),
    ('양파', '15키로', '저장양파', '15키로'),
    ('양파', '12키로', '저장양파', '12키로'),
    ('양파', '20키로', '저장양파', '20키로'),
    ('양파', '12키로', '조생양파', '12키로'),
    ('양파', '15키로', '조생양파', '15키로'),
    ('감자 수미', '20키로상자', '감자', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 대지', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 두백', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 수미(저장)', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 수미(햇)', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 수입', '23키로상자'),
    ('감자 수미', '20키로상자', '감자 조풍', '20키로상자'),
    ('감자 수미', '20키로상자', '홍감자', '10키로상자'),
    ('화건', '30 kg', '햇산화건', '30 kg'),
    ('양건', '30 kg', '햇산양건', '30 kg'),
    ('실파', '20키로', '쪽파', '10키로상자'),
    ('쪽파', '10키로상자', '대파 수입', '10키로묶음'),
    ('쪽파', '10키로상자', '대파(일반)', '10키로묶음'),
    ('쪽파', '10키로상자', '대파(일반)', '1키로단'),
    ('무', '20키로상자', '무', '18키로상자(비닐포)'),
    ('무', '20키로상자', '다발무', '5톤트럭'),
    ('신고', '10 개', '원황', '10 개'),
    ('봄동배추', '15키로상자', '절임배추', '20키로상자'),
    ('쌈배추', '8키로상자', '쌈배추', '1키로상자'),
    ('얼갈이배추', '4키로상자', '얼갈이배추', '8키로상자'),
    ('후지', '10 개', '쓰가루', '10 개')
]

# 가격 컬럼 리스트
price_columns = ['평균가격(원)', '평년 평균가격(원)']

# 평균 비율을 계산하는 함수 (거래단위를 포함)
def calculate_mean_ratio(df1, df2, price_col):
    # '시점', '등급', '거래단위'를 기준으로 inner join
    merged_df = pd.merge(df1[['시점', price_col, '등급', '거래단위']],
                         df2[['시점', price_col, '등급', '거래단위']],
                         on=['시점', '등급', '거래단위'], suffixes=('_1', '_2'), how='inner')

    # 두 품종의 가격이 모두 0이 아닌 경우만 사용
    valid_rows = merged_df[(merged_df[f'{price_col}_1'] != 0) & (merged_df[f'{price_col}_2'] != 0)]

    # 비율 계산 및 평균 비율 구하기
    valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']
    mean_ratio = valid_rows['비율'].mean()

    return mean_ratio

# 0인 값을 대체하는 함수 (평균 비율을 사용)
def replace_zero_in_train(row, mean_ratio, product_1, unit_1, product_2, unit_2, price_col):
    if row['품종명'] == product_1 and row['거래단위'] == unit_1:
        # 같은 시점, 등급, 거래단위의 다른 품종 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) &
                                (train_df['품종명'] == product_2) &
                                (train_df['등급'] == row['등급']) &
                                (train_df['거래단위'] == unit_2)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price * mean_ratio
    elif row['품종명'] == product_2 and row['거래단위'] == unit_2:
        # 같은 시점, 등급, 거래단위의 첫 번째 품종 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) &
                                (train_df['품종명'] == product_1) &
                                (train_df['등급'] == row['등급']) &
                                (train_df['거래단위'] == unit_1)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price / mean_ratio
    return row

# 0 대체 로직 적용 함수
def apply_replacement(product_1, unit_1, product_2, unit_2, price_col, group_by_columns):
    for group_values in train_df[group_by_columns].drop_duplicates().itertuples(index=False):
        # group_by_columns에 따른 필터링
        df1 = train_df[(train_df['품종명'] == product_1) & (train_df['거래단위'] == unit_1) & (train_df[list(group_by_columns)] == group_values).all(axis=1)]
        df2 = train_df[(train_df['품종명'] == product_2) & (train_df['거래단위'] == unit_2) & (train_df[list(group_by_columns)] == group_values).all(axis=1)]

        # 평균 비율 계산
        mean_ratio = calculate_mean_ratio(df1, df2, price_col)
        print(f"{product_1}({unit_1}) - {product_2}({unit_2})의 {price_col}에 대한 평균 비율 {group_by_columns}: {group_values} : {mean_ratio}")

        # 0인 값을 평균 비율로 대체
        train_df.update(train_df.apply(replace_zero_in_train, axis=1, mean_ratio=mean_ratio,
                                       product_1=product_1, unit_1=unit_1,
                                       product_2=product_2, unit_2=unit_2,
                                       price_col=price_col))

# 순차적으로 대체 작업 수행
for product_1, unit_1, product_2, unit_2 in product_pairs:
    for price_col in price_columns:
        # Step 1: 'month'와 '순' 기준으로 대체
        apply_replacement(product_1, unit_1, product_2, unit_2, price_col, ['month', '순'])

        # Step 2: 'month' 기준으로 대체
        apply_replacement(product_1, unit_1, product_2, unit_2, price_col, ['month'])

        # Step 3: 전체 기준으로 대체
        apply_replacement(product_1, unit_1, product_2, unit_2, price_col, [])

# 결과 확인
train_df.head()


양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=1, 순='상순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=1, 순='중순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=1, 순='하순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=2, 순='상순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=2, 순='중순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=2, 순='하순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=3, 순='상순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=3, 순='중순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=3, 순='하순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=4, 순='상순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(month=4, 순='중순') : nan
양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율 ['month', '순']: Pandas(mon

,year,month,순,시점,품목명,품종명,거래단위,등급,평년 평균가격(원),평균가격(원),평년 평균가격(원)_로그,평균가격(원)_로그
0,2018,1,상순,201801상순,건고추,화건,30 kg,상품,381666.666667,590000.0,12.852303,13.287878
1,2018,1,중순,201801중순,건고추,화건,30 kg,상품,380809.666667,590000.0,12.850055,13.287878
2,2018,1,하순,201801하순,건고추,화건,30 kg,상품,380000.000000,590000.0,12.847927,13.287878
3,2018,2,상순,201802상순,건고추,화건,30 kg,상품,380000.000000,590000.0,12.847927,13.287878
4,2018,2,중순,201802중순,건고추,화건,30 kg,상품,376666.666667,590000.0,12.839116,13.287878


In [10]:
import pandas as pd
import re

# 품종 쌍과 거래단위 설정
product_pairs = [
    ('다발무', '5톤트럭', '다발무', '10키로'),
    ('다발무', '5톤트럭', '다발무', '5000키로'),
    ('다발무', '5톤트럭', '다발무', '1000키로'),
    ('다발무', '5톤트럭', '다발무', '10키로'),
    ('다발무', '5톤트럭', '다발무', '8톤트럭')
]

# 가격 및 거래단위 컬럼 리스트
price_columns = ['평균가격(원)', '평년 평균가격(원)']
unit_column = '거래단위_숫자'

# '거래단위'에서 'kg'을 제외한 숫자만 추출하여 새로운 컬럼 '거래단위_숫자' 생성
train_df['거래단위_숫자'] = train_df['거래단위'].apply(lambda x: float(re.sub(r'[^0-9.]', ' ', x)))

# '거래단위'에서 '5톤트럭'과 '8톤트럭' 처리
def process_truck_units(unit):
    if '5톤트럭' in unit:
        return 5000
    elif '8톤트럭' in unit:
        return 8000
    else:
        return float(re.sub(r'[^0-9.]', '', unit))  # 그 외의 경우 숫자만 추출

train_df['거래단위_숫자'] = train_df['거래단위'].apply(process_truck_units)

# 0인 값을 대체하는 함수
def replace_zero_with_unit_price(row, product_1, unit_1, product_2, unit_2, price_col):
    if row['품종명'] == product_1 and row['거래단위'] == unit_1:
        # 같은 시점, 등급의 다른 품종과 거래단위의 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) &
                                (train_df['품종명'] == product_2) &
                                (train_df['등급'] == row['등급']) &
                                (train_df['거래단위'] == unit_2)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                # 첫 번째 품종의 가격이 0이면 두 번째 품종의 가격에 거래단위_숫자를 곱해 단위가격으로 대체
                row[price_col] = other_price * row['거래단위_숫자'] / matching_row['거래단위_숫자'].values[0]
    elif row['품종명'] == product_2 and row['거래단위'] == unit_2:
        # 같은 시점, 등급의 첫 번째 품종과 거래단위의 가격 가져오기
        matching_row = train_df[(train_df['시점'] == row['시점']) &
                                (train_df['품종명'] == product_1) &
                                (train_df['등급'] == row['등급']) &
                                (train_df['거래단위'] == unit_1)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                # 두 번째 품종의 가격이 0이면 첫 번째 품종의 단위가격을 대체
                row[price_col] = (other_price / matching_row['거래단위_숫자'].values[0]) * row['거래단위_숫자']
    return row

# 각 품종 쌍과 가격 컬럼에 대해 처리 적용
for product_1, unit_1, product_2, unit_2 in product_pairs:
    for price_col in price_columns:
        # 0인 값을 단위가격으로 대체
        train_df = train_df.apply(replace_zero_with_unit_price, axis=1,
                                  product_1=product_1, unit_1=unit_1,
                                  product_2=product_2, unit_2=unit_2, price_col=price_col)


In [ ]:
## 코드 확인

import pandas as pd

# 가격 컬럼 리스트
price_columns = ['평균가격(원)', '평년 평균가격(원)']

# 가장 적은 0 값이 있는 등급을 찾는 함수
def get_min_zero_grade(df, price_col):
    return df.groupby('등급').apply(lambda x: (x[price_col] == 0).sum()).idxmin()

# 평균 비율을 계산하는 함수
def calculate_mean_ratio(df1, df2, price_col):
    merged_df = pd.merge(df1[['시점', price_col, '품종명']], df2[['시점', price_col, '품종명']],
                         on=['시점', '품종명'], suffixes=('_1', '_2'), how='inner')

    valid_rows = merged_df[(merged_df[f'{price_col}_1'] != 0) & (merged_df[f'{price_col}_2'] != 0)]
    valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']

    return valid_rows['비율'].mean()

# 0인 값을 대체하는 함수
def replace_zero_in_train(row, mean_ratio, grade_1, grade_2, price_col):
    if row['등급'] == grade_1:
        matching_row = train_df[(train_df['시점'] == row['시점']) & (train_df['등급'] == grade_2) & (train_df['품종명'] == row['품종명'])]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price * mean_ratio
    elif row['등급'] == grade_2:
        matching_row = train_df[(train_df['시점'] == row['시점']) & (train_df['등급'] == grade_1) & (train_df['품종명'] == row['품종명'])]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price / mean_ratio
    return row

# 0 대체 로직을 적용하는 함수
def apply_replacement(product_1, price_col, grade_pairs, group_by_columns):
    for group_values in train_df[group_by_columns].drop_duplicates().itertuples(index=False):
        for grade_1, grade_2 in grade_pairs:
            # 해당 등급과 조건에 맞는 데이터 필터링
            df1 = train_df[(train_df['등급'] == grade_1) & (train_df[list(group_by_columns)] == group_values).all(axis=1) & (train_df['품종명'] == product_1)]
            df2 = train_df[(train_df['등급'] == grade_2) & (train_df[list(group_by_columns)] == group_values).all(axis=1) & (train_df['품종명'] == product_1)]

            # 평균 비율 계산
            mean_ratio = calculate_mean_ratio(df1, df2, price_col)
            print(f"{product_1}의 {grade_1}-{grade_2} {price_col} 평균 비율 {group_by_columns}: {group_values} : {mean_ratio}")

            # 0인 값을 평균 비율로 대체
            train_df.update(train_df.apply(replace_zero_in_train, axis=1, mean_ratio=mean_ratio,
                                           grade_1=grade_1, grade_2=grade_2, price_col=price_col))

# 각 품종별로 가장 적은 0 값이 있는 등급을 기준으로 나머지 등급과 쌍을 이루는 함수
def create_grade_pairs_by_product(df, product, price_col):
    # 품종별 데이터 필터링
    product_df = df[df['품종명'] == product]

    # 기준 등급 (가장 적은 0 값이 있는 등급)
    base_grade = get_min_zero_grade(product_df, price_col)

    # 나머지 등급과 기준 등급으로 쌍 생성
    other_grades = product_df['등급'].unique().tolist()
    other_grades.remove(base_grade)

    return [(base_grade, grade) for grade in other_grades]

# 순차적으로 대체 작업 수행
for product in train_df['품종명'].unique():
    for price_col in price_columns:
        # 품종별 등급 쌍 생성
        grade_pairs = create_grade_pairs_by_product(train_df, product, price_col)

        for grade_pair in grade_pairs:

          # Step 1: 'month'와 '순' 기준으로 대체
          apply_replacement(product, price_col, grade_pairs, ['month', '순'])

          # Step 2: 'month' 기준으로 대체
          apply_replacement(product, price_col, grade_pairs, ['month'])

          # Step 4: 전체 데이터 기준으로 대체
          apply_replacement(product, price_col, grade_pair, [])

# 결과 확인
train_df.head()

화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=1, 순='상순') : 1.1063954517557455
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=1, 순='중순') : 1.1067932109542686
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=1, 순='하순') : 1.107647444852709
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=2, 순='상순') : 1.108105078963648
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=2, 순='중순') : 1.1112440326592465
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=2, 순='하순') : 1.114777184750992
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=3, 순='상순') : 1.1156075105838774
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=3, 순='중순') : 1.114646680114797
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=3, 순='하순') : 1.1101910413684224
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=4, 순='상순') : 1.1038031218705826
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=4, 순='중순') : 1.103111045156131
화건의 상품-중품 평균가격(원) 평균 비율 ['month', '순']: Pandas(month=4, 순='하순') : 1.10

In [ ]:
## 0의 결측값을 스무딩하는 코드

In [ ]:
# 2. 로그 변환된 값을 다시 역변환하여 원래 단위로 복원
train_df['평년 평균가격(원)_복원'] = np.exp(train_df['평년 평균가격(원)_로그'])
train_df['평년 평균가격(원)_복원'] = train_df['평년 평균가격(원)_복원'].fillna(0)

train_df['평균가격(원)_복원'] = np.exp(train_df['평균가격(원)_로그'])
train_df['평균가격(원)_복원'] = train_df['평균가격(원)_복원'].fillna(0)